## sim_rewiring_ex2.py

In [1]:
#!/usr/bin/env python
"""Shows effectiveness of post-spike plasticity."""

import os
import time

from core import core_global as core
from core.spike_monitor import SpikeMonitor
from core.voltage_monitor import VoltageMonitor
from core.weight_matrix_monitor import WeightMatrixMonitor
from layers.rewiring_connection import RewiringConnection
# from layers.rewiring_connection_without_structural_plasticity import RewiringConnection
from models.poisson_pattern_group import PoissonPatternGroup
from utils import utils as utils


def main(args):
    trial = args[0]
    config = args[1]
    input_params = config["input_parameters"]
    connection_params = config["connection_parameters"]
    neuron_params = config["neuron_parameters"]
    
    neuron_params['soma_parameters']['r_l'] = 15
    #### For dendritic threshold check
#     neuron_params['branch_parameters']['thr_dCaAP'] = neuron_params['branch_parameters']['v_thr']
    neuron_params['branch_parameters']['w_dCaAP'] = 40*1.6

    
    print("##############Parameters#############")
    print("simulation_time",config['simulation_time'])
    print("somatic thr" , neuron_params['soma_parameters']['v_thr'])
    print("dendritic thr" , neuron_params['branch_parameters']['thr_dCaAP'])
    

    print("####################################")
    

    exp_result_name_folder = "Random_input_2amp"
    
    ##############################################################

    # Directory for simulation results and log files.
    output_directory = os.path.join("results", "rewiring_ex2",exp_result_name_folder,model_name, time.strftime("%y%m%d_%H%M%S"), str(trial),
                                    "data")

    # Initialize the simulation environment.
    core.init(directory=output_directory)

    # Write config file to the output directory.
    utils.write_configuration(os.path.join(output_directory, "..", "config_rewiring_ex2.yaml"), config)

    # Set the random seed.
    core.kernel.set_master_seed(config["master_seed"])

    # Create input neurons.
    inp = PoissonPatternGroup(input_params["num_inputs"], input_params["rate"], input_params["rate_bg"],
                              params=input_params)

    # Create the neuron.
    neuron = McLifGroup(1, neuron_params["num_branches"], neuron_params)
#     neuron.it_is_working

    # Connect input to neuron.
    conn = RewiringConnection(inp, neuron, neuron.branch.syn_current, connection_params)

    # Create some monitors which will record the simulation data.
    WeightMatrixMonitor(conn, core.kernel.fn("weights", "dat"),
                        interval=config["sampling_interval_weights"])
    SpikeMonitor(neuron, core.kernel.fn("output", "ras"))
    sm_inp = SpikeMonitor(inp, core.kernel.fn("input", "ras"))
    vm_nrn = VoltageMonitor(neuron.soma, 0, core.kernel.fn("soma", "mem"),paste_spikes=True, paste_spikes_from=neuron)
    vm_br = []
    for i in range(neuron_params["num_branches"]):
        vm_br.append(VoltageMonitor(neuron.branch, (i, 0), core.kernel.fn("branch", "mem", i)))


    # Now simulate the model.
    simulation_time = config["simulation_time"]
    core.kernel.run_chunk(20.0, 0, simulation_time)

    sm_inp.active = False
    vm_nrn.active = False
    for vm in vm_br:
        vm.active = False
    core.kernel.run_chunk(simulation_time - 40, 0, simulation_time)

    sm_inp.active = True
    vm_nrn.active = True
    for vm in vm_br:
        vm.active = True
    core.kernel.run_chunk(20.0, 0, simulation_time)


## Long Run

In [2]:
# for model_name in ['Gidon_dend','Limbacher']:
for model_name in ['Gidon_dend']:

    if model_name == 'Limbacher':
        from models.mc_lif_group import McLifGroup
    elif model_name == 'Gidon_dend':
        from models.Gidon_mc_lif_group_current import McLifGroup

    print(model_name)
################################################################    
    if __name__ == '__main__':
        import copy
        from scoop import futures

        # Load the configuration file.
        config = utils.load_configuration("config_rewiring_ex2.yaml")

        configs = []
        num_trials = 10
        for trial in range(num_trials):
            config["master_seed"] = 10 * (trial + 1)
            configs.append(copy.deepcopy(config))

        r = list(futures.map(main, [[trial, config] for trial, config in enumerate(configs)]))

Gidon_dend
##############Parameters#############
simulation_time 1000.0
somatic thr -55.0
dendritic thr -36.0
####################################
Gidon_dendrites
Graded:  True
dendrite thr: -36.0
Soma thr: -55.0


/Users/sima/anaconda3/lib/python3.10/site-packages/scoop/fallbacks.py:38: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().
  warnings.warn(


##############Parameters#############
simulation_time 1000.0
somatic thr -55.0
dendritic thr -36.0
####################################
Gidon_dendrites
Graded:  True
dendrite thr: -36.0
Soma thr: -55.0
##############Parameters#############
simulation_time 1000.0
somatic thr -55.0
dendritic thr -36.0
####################################
Gidon_dendrites
Graded:  True
dendrite thr: -36.0
Soma thr: -55.0
##############Parameters#############
simulation_time 1000.0
somatic thr -55.0
dendritic thr -36.0
####################################
Gidon_dendrites
Graded:  True
dendrite thr: -36.0
Soma thr: -55.0
##############Parameters#############
simulation_time 1000.0
somatic thr -55.0
dendritic thr -36.0
####################################
Gidon_dendrites
Graded:  True
dendrite thr: -36.0
Soma thr: -55.0
##############Parameters#############
simulation_time 1000.0
somatic thr -55.0
dendritic thr -36.0
####################################
Gidon_dendrites
Graded:  True
dendrite thr: -36.0
Soma t